# CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import metrics
import numpy as np

# Define image dimensions and other hyperparameters
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 16
epochs = 50
num_classes = 4

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:

# Compile the model with appropriate optimizer, loss function, and evaluation metric
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.Recall()])

# Create an ImageDataGenerator for data augmentation and image preprocessing
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation data
train_generator = train_datagen.flow_from_directory(
        'brain_tumour/train/',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'brain_tumour/test/',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Train the model
history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size)

In [ ]:
# Save the trained model
model.save('braintumour_model.h5')

In [ ]:
# Get predictions and true labels for the test set
test_generator = test_datagen.flow_from_directory(
        'test/',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Y_pred = model.predict(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

In [ ]:
# Calculate precision, recall, and accuracy for each class
class_names = list(test_generator.class_indices.keys())
print("Class Names: ", class_names)
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# ML Algorithms

In [39]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform

# Define the paths to your training and testing image directories
train_dir = 'brain_tumour/train/'
test_dir = 'brain_tumour/test/'

# Load and preprocess image data
def load_and_preprocess_images(directory):
    images = []
    labels = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            image = io.imread(file_path)
            resized_image = transform.resize(image, (224, 224, 3))  # Resize to desired size
            images.append(resized_image.flatten())  # Flatten the image
            labels.append(category)
    return np.array(images), np.array(labels)


In [40]:
X_train, y_train = load_and_preprocess_images(train_dir)
X_test, y_test = load_and_preprocess_images(test_dir)

In [41]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [42]:
print("X_train shape:", X_train.shape)
print("y_train_encoded shape:", y_train_encoded.shape)
print("X_test shape:", X_test.shape)
print("y_test_encoded shape:", y_test_encoded.shape)

X_train shape: (5712, 150528)
y_train_encoded shape: (5712,)
X_test shape: (1311, 150528)
y_test_encoded shape: (1311,)


In [43]:
# Initialize and train models
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train_encoded)

RandomForestClassifier(random_state=42)

In [57]:
# Logistic regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train_encoded)

c:\Users\z035793\Documents\others\side_projects\braintumour-classifier\env\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [63]:
# KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train_encoded)

KNeighborsClassifier()

In [64]:
# Decision tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train_encoded)

DecisionTreeClassifier(random_state=42)

In [70]:
# SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train_encoded)

SVC(kernel='linear')

In [71]:
# Predict on the test set
rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)
dt_predictions = dt_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
svm_predictions = svm_model.predict(X_test)

In [73]:
# Evaluate models
def evaluate_model(predictions, model_name):
    predictions = label_encoder.inverse_transform(predictions)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    print(f"Results for {model_name}:\n")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{report}\n")

# Evaluation Results

In [74]:
evaluate_model(rf_predictions, "Random Forest")

Results for Random Forest:

Accuracy: 0.94
Classification Report:
              precision    recall  f1-score   support

      glioma       0.97      0.84      0.90       300
  meningioma       0.85      0.93      0.89       306
     notumor       0.98      1.00      0.99       405
   pituitary       0.96      0.97      0.97       300

    accuracy                           0.94      1311
   macro avg       0.94      0.94      0.94      1311
weighted avg       0.94      0.94      0.94      1311




In [75]:
evaluate_model(lr_predictions, "Logistic Regression")

Results for Logistic Regression:

Accuracy: 0.90
Classification Report:
              precision    recall  f1-score   support

      glioma       0.89      0.80      0.84       300
  meningioma       0.80      0.84      0.82       306
     notumor       0.96      0.99      0.97       405
   pituitary       0.95      0.95      0.95       300

    accuracy                           0.90      1311
   macro avg       0.90      0.90      0.90      1311
weighted avg       0.90      0.90      0.90      1311




In [76]:
evaluate_model(knn_predictions, "k-Nearest Neighbors")

Results for k-Nearest Neighbors:

Accuracy: 0.85
Classification Report:
              precision    recall  f1-score   support

      glioma       0.77      0.93      0.84       300
  meningioma       0.89      0.52      0.66       306
     notumor       0.87      0.94      0.90       405
   pituitary       0.89      0.99      0.93       300

    accuracy                           0.85      1311
   macro avg       0.85      0.84      0.83      1311
weighted avg       0.86      0.85      0.84      1311




In [77]:
evaluate_model(dt_predictions, "Decision Tree")

Results for Decision Tree:

Accuracy: 0.88
Classification Report:
              precision    recall  f1-score   support

      glioma       0.88      0.83      0.85       300
  meningioma       0.77      0.84      0.80       306
     notumor       0.97      0.96      0.96       405
   pituitary       0.90      0.89      0.90       300

    accuracy                           0.88      1311
   macro avg       0.88      0.88      0.88      1311
weighted avg       0.89      0.88      0.89      1311




In [78]:
evaluate_model(svm_predictions, "SVM")

Results for SVM:

Accuracy: 0.91
Classification Report:
              precision    recall  f1-score   support

      glioma       0.85      0.82      0.84       300
  meningioma       0.82      0.83      0.82       306
     notumor       0.98      0.99      0.99       405
   pituitary       0.97      0.98      0.97       300

    accuracy                           0.91      1311
   macro avg       0.91      0.90      0.91      1311
weighted avg       0.91      0.91      0.91      1311




# ResNet

In [81]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder("brain_tumour/train", transform=transform)
test_dataset = datasets.ImageFolder("brain_tumour/test", transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the ResNet model
model = models.resnet18(pretrained=True)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(correct / total) * 100:.2f}%")


Epoch [1/10], Loss: 0.2450
Epoch [2/10], Loss: 0.3225
Epoch [3/10], Loss: 0.1615
Epoch [4/10], Loss: 0.0779
Epoch [5/10], Loss: 0.0114
Epoch [6/10], Loss: 0.0512
Epoch [7/10], Loss: 0.0054
Epoch [8/10], Loss: 0.0075
Epoch [9/10], Loss: 0.0009
Epoch [10/10], Loss: 0.1805
Test Accuracy: 96.49%


In [82]:
from sklearn.metrics import classification_report

# Evaluation
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=train_dataset.classes))

Classification Report:
              precision    recall  f1-score   support

      glioma       0.92      0.97      0.94       300
  meningioma       0.95      0.94      0.95       306
     notumor       1.00      1.00      1.00       405
   pituitary       0.99      0.94      0.96       300

    accuracy                           0.96      1311
   macro avg       0.96      0.96      0.96      1311
weighted avg       0.97      0.96      0.97      1311



# VGG Model

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder("brain_tumour/train", transform=transform)
test_dataset = datasets.ImageFolder("brain_tumour/test", transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the VGG model
model = models.vgg16(pretrained=True)
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

c:\Users\z035793\Documents\others\side_projects\braintumour-classifier\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\z035793\Documents\others\side_projects\braintumour-classifier\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    avg_train_loss = train_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss:.4f}")

Epoch [1/10], Average Training Loss: 1.0079
Epoch [2/10], Average Training Loss: 0.6135
Epoch [3/10], Average Training Loss: 0.4787
Epoch [4/10], Average Training Loss: 0.4085
Epoch [5/10], Average Training Loss: 0.3653


In [ ]:
# Evaluation
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate classification report
target_names = test_dataset.classes
classification_rep = classification_report(all_labels, all_predictions, target_names=target_names)

print("Classification Report:")
print(classification_rep)